# Convert netCDF to Zarr format

In [1]:
## import required packages
import numpy as np
import xarray as xr
import s3fs
# import pandas as pd
# from datetime import datetime
# import json
# from contextlib import contextmanager
import argparse
from pathlib import Path
import os
# import time 
import netCDF4 as nc4

In [2]:
sassie_key = 
sassie_secret = 

In [13]:
sassie_s3_netcdf_dir = 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/'
var_name = 'THETA'
var_name = 'SIarea'

In [29]:
files_to_process = [0,2]

In [30]:
## initialize s3 system for sassie bucket
s3 = s3fs.S3FileSystem(anon=False, key=sassie_key, secret=sassie_secret) 

## list all files
nc_file_list = np.sort(s3.glob(f'{sassie_s3_netcdf_dir}{var_name}_AVG_DAILY/*.nc'))
nc_file_list = np.sort(s3.glob(f'{sassie_s3_netcdf_dir}{var_name}_AVG_DAILY/*.nc'))

In [31]:
## append "s3://" to create url in order to open the dataset
nc_file_list_urls = []
for file in nc_file_list:
    file_url_tmp = f"s3://{file}"
    nc_file_list_urls.append(file_url_tmp)

In [32]:
nc_file_list_urls[0:4]

['s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc',
 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-16_ECCO_SASSIE_V1_HH_llc1080.nc',
 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-17_ECCO_SASSIE_V1_HH_llc1080.nc',
 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-18_ECCO_SASSIE_V1_HH_llc1080.nc']

In [33]:
## specify start and end indices or process all files   
if len(files_to_process) == 2: # two numbers indicates a range (two indices)
    data_urls_select = nc_file_list_urls[files_to_process[0]:files_to_process[1]]
    print(f'... first file to process : {data_urls_select[0]}')
    print(f'... last file to process  : {data_urls_select[-1]}')

elif len(files_to_process) == 1 and files_to_process[0] == -1: # process all files
    data_urls_select = nc_file_list_urls
    print(f'... first file to process : {data_urls_select[0]}')
    print(f'... last file to process  : {data_urls_select[-1]}')

elif len(files_to_process) == 1 and files_to_process[0] >= 0: # process one file using number as index
    # wrap in list
    data_urls_select = [nc_file_list_urls[files_to_process[0]]]
    print(f'... 1 file to process : {data_urls_select}')

else:
    print("ERROR: invalid entry for `files_to_process` argument")

... first file to process : s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc
... last file to process  : s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-16_ECCO_SASSIE_V1_HH_llc1080.nc


In [ ]:
zarr_s3_bucket_dir = 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/ZARR/'

In [22]:
## loop through each file and save new zarr format
for file_url in data_urls_select:
    
    print(f"\n... opening {file_url}")
    s3_file = s3.open(file_url)
    s3_file_ec2 = xr.open_dataset(s3_file)
    s3_file_ec2.close()

    ## mode='a' means append so it will append each netcdf dataset to the zarr store
    s3_file_ec2.to_zarr('tmp.zarr', mode='a', encoding={'compressor': 'zlib', 'level': 5})
    
    # ## edit filename to indicate that it is a surface layer file (not 3D)
    # filename_split = file_url.split("/")[-1].split("day")
    # netcdf_filename_new = f"{filename_split[0]}SURFACE_day{filename_split[1]}"
    # ## create encoding
    # encoding_var = create_encoding(tmp_surface, output_array_precision = np.float32)
    
    # tmp_surface.to_netcdf(f"{ec2_scratch_dir}/{netcdf_filename_new}", encoding = encoding_var)
    # tmp_surface.close()
    # print(f"* * * * saved netcdf to {ec2_scratch_dir}/{netcdf_filename_new} * * * *\n")


... opening s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/SIarea_AVG_DAILY/SIarea_day_mean_2014-01-15_ECCO_SASSIE_V1_HH_llc1080.nc


#### on the cloud

In [ ]:
s3_store = s3fs.S3Map(root=zarr_s3_bucket_dir, s3=s3, check=False)
zarr_dataset = zarr.group(store=s3_store)

In [26]:
## open zarr
tmp = xr.open_zarr('tmp.zarr')

In [27]:
tmp

<xarray.Dataset> Size: 31MB
Dimensions:    (time: 1, j: 1080, i: 1800, nv: 2)
Coordinates:
    XC         (j, i) float32 8MB dask.array<chunksize=(270, 450), meta=np.ndarray>
    YC         (j, i) float32 8MB dask.array<chunksize=(270, 450), meta=np.ndarray>
  * i          (i) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
  * time       (time) datetime64[ns] 8B 2014-01-15T12:00:00
    time_bnds  (time, nv) datetime64[ns] 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables:
    SIarea     (time, j, i) float64 16MB dask.array<chunksize=(1, 135, 450), meta=np.ndarray>
Attributes: (12/51)
    Conventions:               CF-1.8, ACDD-1.3
    NCO:                       netCDF Operators version 5.2.1 (Homepage = htt...
    acknowledgement:           This research was carried out by the Jet Propu...
    author:                    Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:             Grid
    comment:                   SASSIE llc1080 V1R1 fields are consolidated on...
    ...                        ...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   This dataset provides data variable and geomet...
    time_coverage_end:         2014-01-15T00:00:00
    time_coverage_start:       2014-01-16T00:00:00
    title:                     SASSIE Ocean Model SIarea Parameter for the La...
    uuid:                      852b1f3e-ea49-11ee-bb1f-06f6cde23aa5